## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bethel,US,41.3712,-73.4140,64.02,78,65,1.01,broken clouds
1,1,Souillac,MU,-20.5167,57.5167,74.35,73,21,5.46,few clouds
2,2,Palmer,US,42.1584,-72.3287,64.72,67,83,1.99,broken clouds
3,3,Mosquera,CO,4.7059,-74.2302,62.69,72,75,6.91,broken clouds
4,4,Lebu,CL,-37.6167,-73.6500,60.12,72,100,7.96,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?70
What is the maximum temperature you would like for your trip?85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Souillac,MU,-20.5167,57.5167,74.35,73,21,5.46,few clouds
5,5,Atuona,PF,-9.8000,-139.0333,80.08,67,95,13.38,overcast clouds
7,7,Kroya,ID,-7.6331,109.2461,78.35,87,94,2.80,heavy intensity rain
8,8,Butaritari,KI,3.0707,172.7902,80.85,78,37,9.31,light rain
11,11,Rikitea,PF,-23.1203,-134.9692,74.05,88,100,18.12,light rain
19,19,Cozumel,MX,20.5083,-86.9458,82.99,54,20,3.00,few clouds
21,21,Harper,LR,4.3750,-7.7169,82.20,74,67,10.65,light rain
22,22,Vaini,TO,-21.2000,-175.2000,71.76,94,20,5.75,few clouds
23,23,Ternate,ID,0.8000,127.4000,80.73,79,100,3.56,overcast clouds
24,24,Payo,ID,-3.7508,103.6372,73.20,98,100,3.42,heavy intensity rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Souillac,MU,74.35,few clouds,-20.5167,57.5167,
5,Atuona,PF,80.08,overcast clouds,-9.8000,-139.0333,
7,Kroya,ID,78.35,heavy intensity rain,-7.6331,109.2461,
8,Butaritari,KI,80.85,light rain,3.0707,172.7902,
11,Rikitea,PF,74.05,light rain,-23.1203,-134.9692,
19,Cozumel,MX,82.99,few clouds,20.5083,-86.9458,
21,Harper,LR,82.20,light rain,4.3750,-7.7169,
22,Vaini,TO,71.76,few clouds,-21.2000,-175.2000,
23,Ternate,ID,80.73,overcast clouds,0.8000,127.4000,
24,Payo,ID,73.20,heavy intensity rain,-3.7508,103.6372,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping.")
        

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(3)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Souillac,MU,74.35,few clouds,-20.5167,57.5167,Shanti Maurice Resort & Spa
5,Atuona,PF,80.08,overcast clouds,-9.8000,-139.0333,Villa Enata
7,Kroya,ID,78.35,heavy intensity rain,-7.6331,109.2461,RUSTANTO Fam


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig = gmaps.figure(center = (30.0, 31.0), zoom_level = 3)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))